#### Active learning NNGP experiment
We use one set of params for simple example

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error as mse
import matplotlib.pyplot as plt

from model.mlp import MLP
from dataloader.rosen import RosenData
from uncertainty_estimator.nngp import NNGP
from sample_selector.eager import EagerSampleSelector
from oracle.identity import IdentityOracle


In [ ]:
config = {
    'random_state': 4623457,
    'n_dim': 10,
    'n_train': 200,
    'n_test': 200,
    'n_pool': 1000,
    'layers': [128, 64, 32],
    'update_sample_size': 10,
    'al_iterations': 10
}

np.random.seed(config['random_state'])

In [ ]:
def print_shapes(note, *sets):
    print(note)
    for x, y in sets:
        print("shapes:", x.shape, y.shape)

# load data
X_train, y_train, _, _, X_test, y_test, X_pool, y_pool = RosenData(
    config['n_train'], 0, config['n_test'], config['n_pool'], config['n_dim']
).dataset()

In [ ]:
# Init neural network & tf session
tf.reset_default_graph()

model = MLP(
    ndim = config['n_dim'],
    random_state = config['random_state'],
    layers = config['layers']
)

try:
    sess.close()
except:
    pass
session_config = tf.ConfigProto()
session_config.gpu_options.allow_growth = True

init = tf.global_variables_initializer()
sess = tf.Session(config=session_config)
sess.run(init)

In [ ]:
estimator = NNGP(model)  # to estimate uncertainties
oracle = IdentityOracle(y_pool)  # generate y for X from pool
sampler = EagerSampleSelector(oracle) # sample X and y from pool by uncertainty estimations

In [ ]:
model.train(sess, X_train, y_train, X_test, y_test, X_test, y_test)

rmses = [np.sqrt(mse(model.predict(sess, data=X_test), y_test))]


for al_iteration in range(1, config['al_iterations']+1):
    note = f'[{al_iteration}] BEFORE:'
    print_shapes(note, (X_train, y_train), (X_test, y_test), (X_pool, y_pool))
    
    # update pool
    uncertainties = estimator.estimate(sess, X_train, y_train, X_pool)
    X_train, y_train, X_pool = sampler.update_sets(
        X_train, y_train, X_pool, uncertainties, config['update_sample_size']
    )
    
    note = f'[{al_iteration}] AFTER:'
    print_shapes(note, (X_train, y_train), (X_test, y_test), (X_pool, y_pool))
    
    # retrain net
    model.train(sess, X_train, y_train, X_test, y_test, X_test, y_test)
    rmses.append(np.sqrt(mse(model.predict(sess, data=X_test), y_test)))

In [ ]:
plt.plot(rmses, label='NNGP', marker='.')
plt.title('RMS Error by active learning iterations')
plt.legend()

In [ ]:
# Show some predictions
print(model.predict(sess, data = X_test[:3]))
print(y_test[:3])

In [ ]:
# show some uncertainties
estimator.estimate(sess, X_train, y_train, X_pool)